<a href="https://colab.research.google.com/github/nishithewel/ID5059GroupProject/blob/main/ID5059_ieee_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!from google.colab import files
!files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"visheshb","key":"d68d982088da656c18e60446755a109d"}'}

In [ ]:
! rm -rf sample_data
!mkdir  -p /root/.kaggle/
!mv kaggle.json  /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
! ls -lart /root/.kaggle/
!kaggle competitions download -c 'ieee-fraud-detection'
! mkdir data
! mv ieee-fraud-detection.zip data/
! unzip data/ieee-fraud-detection.zip -d data/

mv: cannot stat 'kaggle.json': No such file or directory
total 16
-rw------- 1 root root   64 Mar 18 09:36 kaggle.json
drwx------ 1 root root 4096 Mar 18 09:41 ..
drwxr-xr-x 2 root root 4096 Mar 18 09:46 .


# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

 82% 97.0M/118M [00:01<00:00, 107MB/s] 
100% 118M/118M [00:01<00:00, 113MB/s] 


# New Section

# New Section